In [ ]:
# Data: http://saludata.saludcapital.gov.co/osb/datos_abiertos_osb/enf-transmisibles/OSB_EnfTransm-Uso-UCIS-COVID.csv

In [ ]:
import requests
import pandas as pd
import re
import numpy
from google.cloud import storage

In [ ]:
URL = 'http://saludata.saludcapital.gov.co/osb/datos_abiertos_osb/enf-transmisibles/OSB_EnfTransm-Uso-UCIS-COVID.csv'
FILEPATH = './data/uci_bogota_raw.csv'

In [ ]:
r = requests.get( URL, stream = True )

with open( FILEPATH, 'wb' ) as f:
    f.write( r.content )

In [ ]:
ucis = pd.read_csv( './data/uci_bogota_raw.csv', encoding = 'ISO-8859-1', sep = ';', skiprows = 4, skipfooter = 3 )

In [ ]:
def convert_int64(column:str):
    temp=ucis[column]
    i=0
    for row in temp:
        data=str(row) 
        search=re.findall('^[1-9]\.\w*',data)
        if(len(search)==1):
            #Select first item in list of re
            str1=search[0]
            
            #To float to aproximate 3 digits
            str1=float(str1)
            str1=round(str1,3)
            #Cast as str and replace '.'
            str1=str(str1).replace('.','')
            if(len(str1)==3):
                str1=str1+'0'
                
            #Replace in the column
            temp[i]=str1
            temp[i]=temp[i].astype(numpy.int64)
            
        else:
            row=float(data)
            temp[i]=int(row)
            temp[i]=temp[i].astype(numpy.int64)
        i+=1
    
    ucis[column]=temp.astype(numpy.int64)

convert_int64('Camas UCI ocupadas Covid-19')
convert_int64('Camas UCI Disponibles COVID 19')

In [ ]:
ucis.shape

In [ ]:
ucis.dtypes

In [ ]:
ucis[ 'Fecha' ] = pd.to_datetime( ucis[ 'Fecha' ], format = '%d/%m/%Y' )

In [ ]:
ucis[ '% Ocupación' ] = ucis[ 'Camas UCI ocupadas Covid-19' ] / ucis[ 'Camas UCI Disponibles COVID 19' ]

In [ ]:
ucis.dtypes

In [ ]:
del ucis[ 'Ocupación UCI COVID 19' ]

In [ ]:
ucis.rename( columns = { '% Ocupación': 'Ocupacion' }, inplace = True )

In [ ]:
ucis.tail( 20 )

In [ ]:
#ucis.to_csv( './data/uci_bogota.csv', index = False )

In [ ]:
#DataFrame to CSV
csv_data=ucis.to_csv(index=False)

#Client, bucket and blob to acces gcloud
client= storage.Client()
bucket= client.get_bucket('siscovid')
blob=bucket.blob('uci_bogota.csv')

#Uploads the file to gcloud
blob.upload_from_string(data=csv_data, content_type='text/csv')